# 집값 예측 - GPU 서버 파이프라인

이 노트북은 GPU 서버 환경에서 실행되도록 설계되었습니다.
API를 이용한 좌표 보완과 **KNN 결측치 보완**을 포함한 전체 전처리 과정을 수행합니다.

## 설정 및 경로 구성
다음과 같은 디렉토리 구조를 확인하거나, 아래 코드에서 `DATA_DIR`을 알맞게 수정하세요:

```
project_root/
├── 병합/ (현재 디렉토리)
│   └── run_pipeline_gpu.ipynb
├── data/
│   └── raw/
│       ├── train.csv
│       └── test.csv
```

> **참고:** 서버에 `rapids` (cuML) 라이브러리가 설치되어 있다면, `sklearn.impute.KNNImputer` 대신 `cuml.preprocessing.KNNImputer`를 사용하여 GPU 가속을 활용할 수 있습니다.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import requests
import time
from tqdm import tqdm
from sklearn.impute import KNNImputer

# GPU 사용 가능 여부 확인 (선택 사항 - 정보 확인용)
try:
    import torch
    if torch.cuda.is_available():
        print(f"[정보] GPU 감지됨: {torch.cuda.get_device_name(0)}")
    else:
        print("[정보] GPU가 감지되지 않았습니다. CPU로 실행합니다.")
except ImportError:
    print("[정보] PyTorch가 설치되지 않았습니다. (GPU 확인 생략)")

# ==========================================
# 1. 데이터 경로 자동 탐지
# ==========================================
# 다양한 환경(로컬, 서버 등)에 대응하기 위해 여러 예상 경로를 확인합니다.
POSSIBLE_PATHS = [
    '../data/raw',       # 표준 프로젝트 구조
    'data/raw',          # 실행 폴더 내에 data가 있는 경우
    '../../data/raw',    # 더 깊은 경로에 있는 경우
    './'                 # 현재 디렉토리에 파일이 있는 경우
]

DATA_DIR = None
for path in POSSIBLE_PATHS:
    if os.path.exists(os.path.join(path, 'train.csv')):
        DATA_DIR = path
        break

if DATA_DIR is None:
    raise FileNotFoundError("표준 경로에서 'train.csv'를 찾을 수 없습니다. DATA_DIR을 수동으로 설정해주세요.")

print(f"데이터 디렉토리 확인됨: {DATA_DIR}")
TRAIN_PATH = os.path.join(DATA_DIR, 'train.csv')
TEST_PATH = os.path.join(DATA_DIR, 'test.csv')

# ==========================================
# 2. API 설정
# ==========================================

# 보안을 위해 환경 변수에서 먼저 API 키를 로드합니다.
KAKAO_API_KEY = os.environ.get("KAKAO_API_KEY")

# 환경 변수에 키가 없을 경우, 하드코딩된 키를 사용합니다. (테스트 편의성)
if not KAKAO_API_KEY:
    KAKAO_API_KEY = "50721163f60b5e5c192f6c3847602b05"  
    print("경고: 하드코딩된 API 키를 사용합니다. (유효한 키인지 확인 필요)")
else:
    print("환경 변수에서 API 키를 로드했습니다.")


## 단계 1: 취소된 거래 제거

In [ ]:
def step1_filter_cancelled(df):
    """
    취소된 거래(해제사유발생일이 존재하는 데이터)를 제거합니다.
    """
    print("[단계 1] 취소된 거래 필터링 중...")
    if '해제사유발생일' in df.columns:
        n_cancelled = df['해제사유발생일'].notnull().sum()
        if n_cancelled > 0:
            print(f" -> {n_cancelled}건의 취소된 거래를 발견했습니다. 제거합니다...")
            # 해제사유발생일이 비어있는(정상 거래) 데이터만 유지
            df = df[df['해제사유발생일'].isnull()].copy()
            # 컬럼 삭제
            df.drop(columns=['해제사유발생일'], inplace=True)
    return df

# 데이터 로드
print("데이터 로딩 중...")
df_train = pd.read_csv(TRAIN_PATH)
print(f"학습 데이터 크기: {df_train.shape}")

df_test = pd.read_csv(TEST_PATH)
print(f"테스트 데이터 크기: {df_test.shape}")

df_train = step1_filter_cancelled(df_train)
df_test = step1_filter_cancelled(df_test)

## 단계 2~7: 전처리 파이프라인 (타입 변환, 클리닝, 특성 공학)

In [ ]:
def preprocessing_pipeline(df):
    """
    기본적인 전처리 과정을 순차적으로 수행합니다.
    1. 자료형 변환 (Float -> Int64)
    2. 날짜 파싱 (계약년월 + 계약일 -> 계약일자)
    3. 주소 파싱 및 지번주소 생성
    4. 불필요한 컬럼 제거
    """
    
    # 1. 자료형 변환 (메모리 최적화를 위해 Float을 Int64로 변환)
    int_cols = [
        '본번', '부번', 'k-전체동수', 'k-전체세대수', '주차대수',
        'k-연면적', 'k-주거전용면적', 'k-관리비부과면적',
        'k-전용면적별세대현황(60㎡이하)', 'k-전용면적별세대현황(60㎡~85㎡이하)', 
        'k-85㎡~135㎡이하', 'k-135㎡초과'
    ]
    for col in int_cols:
        if col in df.columns:
            try:
                # 반올림 후 Int64(Nullable Int)로 변환
                df[col] = df[col].round().astype('Int64')
            except: pass

    # 2. 날짜 파싱
    if '계약년월' in df.columns and '계약일' in df.columns:
        # 계약년월(202301) + 계약일(1 -> 01) 합쳐서 하나의 정수형 날짜(20230101)로 생성
        df['계약일자'] = (df['계약년월'].astype(str) + df['계약일'].astype(str).str.zfill(2)).astype('Int64')
        df.drop(columns=['계약일'], inplace=True, errors='ignore')

    # 3. 주소 파싱 (시군구 분리 및 지번주소 재조합)
    if '시군구' in df.columns:
        split = df['시군구'].str.split(' ', expand=True)
        if split.shape[1] >= 2: df['구'] = split[1]
        if split.shape[1] >= 3: df['동'] = split[2]
    
    if '본번' in df.columns and '부번' in df.columns:
        try:
            # 결측치를 0으로 채우고 문자열로 변환
            bon = df['본번'].fillna(0).astype(int).astype(str)
            bu = df['부번'].fillna(0).astype(int).astype(str)
            
            # 벡터화 연산으로 주소 조합 (시군구 + 본번 + [-부번])
            full_addr = df['시군구'] + " " + bon
            mask_bu = (df['부번'].fillna(0) > 0)
            full_addr.loc[mask_bu] += "-" + bu.loc[mask_bu]
            
            df['지번주소'] = full_addr
            # 사용된 원본 컬럼 제거
            df.drop(columns=['본번', '부번'], inplace=True)
        except Exception as e:
            print(f"주소 생성 중 경고 발생: {e}")

    # 4. 불필요한 컬럼 제거
    # 분석 결과 유의미하지 않거나 정보가 부족한 컬럼들
    drop_cols = [
        'k-등록일자', 'k-홈페이지', 'k-수정일자', 'k-전화번호', 'k-팩스번호',
        '고용보험관리번호', '단지신청일', '단지승인일', '사용허가여부', '관리비 업로드',
        '기타/의무/임대/임의=1/2/3/4', '단지소개기존clob', '건축면적',
        '계약년월', 'k-주거전용면적', '번지', '시군구',
        '중개사소재지', '등기신청일자', '도로명', 'k-시행사', 
        'k-단지분류(아파트,주상복합등등)', 'k-관리방식',
        'k-전용면적별세대현황(60㎡이하)', 'k-전용면적별세대현황(60㎡~85㎡이하)',
        'k-85㎡~135㎡이하', 'k-135㎡초과'
    ]
    df.drop(columns=[c for c in drop_cols if c in df.columns], inplace=True)

    # 컬럼명 정리
    rename_map = {'k-사용검사일-사용승인일': '사용검사일'}
    df.rename(columns=rename_map, inplace=True)

    return df

print("전처리 파이프라인 적용 중...")
df_train = preprocessing_pipeline(df_train)
df_test = preprocessing_pipeline(df_test)
print("완료.")

## 단계 8 & 9: 좌표 보완 (API 이용)
**참고:** API 속도가 느리거나 제한에 걸린다면, 이 단계를 중단하고 다음 단계로 넘어가세요. (중단 시점까지의 데이터는 메모리에 남아있습니다)

In [ ]:
def get_coords(addr):
    """카카오 API를 사용하여 주소의 좌표를 가져옵니다."""
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    try:
        res = requests.get(url, headers=headers, params={"query": addr}, timeout=3)
        if res.status_code == 200:
            docs = res.json().get('documents')
            if docs: return float(docs[0]['x']), float(docs[0]['y'])
    except:
        pass
    return None, None

def fill_coordinates(df):
    """결측된 좌표 정보를 API를 통해 보완합니다."""
    print("좌표 보완 작업 시작...")
    
    # 결측치 확인
    missing = df[(df['좌표X'].isnull()) | (df['좌표X'] == 0)]
    if len(missing) == 0:
        print(" -> 결측된 좌표가 없습니다.")
        return df
        
    print(f" -> 결측 건수: {len(missing)} 건")
    unique_addr = missing['지번주소'].unique()
    cache = {}
    
    # 일괄 처리 (API 호출)
    for addr in tqdm(unique_addr, desc="API 호출 중"):
        if pd.isna(addr): continue
        cache[addr] = get_coords(str(addr))
        
    # 결과 적용
    for idx in missing.index:
        addr = df.at[idx, '지번주소']
        x, y = cache.get(addr, (None, None))
        if x:
            df.at[idx, '좌표X'] = x
            df.at[idx, '좌표Y'] = y
            
    return df

# 실행
print("학습 데이터 좌표 보완 중...")
df_train = fill_coordinates(df_train)
print("테스트 데이터 좌표 보완 중...")
df_test = fill_coordinates(df_test)

## 단계 10: KNN 결측치 보완 (좌표 기반)

**GPU 가속 팁:**
만약 `cuml` 라이브러리(RAPIDS)가 설치되어 있다면, 아래 코드를 사용하여 더 빠르게 실행할 수 있습니다:
```python
from cuml.preprocessing import KNNImputer
```
그렇지 않은 경우 일반 `sklearn` (CPU) 방식이 사용됩니다.

In [ ]:
def run_knn_imputation(df, k=5):
    """
    좌표(X, Y) 정보를 활용하여 이웃한 아파트의 정보로 수치형 결측치를 보완합니다.
    """
    print("[KNN] 결측치 보완 작업 시작...")
    
    # 보완할 대상 컬럼 (타겟)
    target_cols = ['주차대수', '건축년도', 'k-전체세대수', 'k-연면적']
    # 거리 계산에 사용할 컬럼 (피쳐)
    feature_cols = ['좌표X', '좌표Y']
    
    # KNN 작동을 위해 좌표가 반드시 있어야 합니다. 
    # 여전히 결측인 좌표는 평균값으로 임시 대치합니다.
    for c in feature_cols:
        if df[c].isnull().sum() > 0:
            print(f" -> {c} 컬럼에 남은 결측치가 있어 평균값으로 대치합니다. (KNN 안전장치)")
            df[c].fillna(df[c].mean(), inplace=True)
            
    # 실제 데이터프레임에 존재하는 타겟 컬럼만 선택
    valid_targets = [c for c in target_cols if c in df.columns]
    if not valid_targets: return df
    
    # 데이터 준비
    use_cols = feature_cols + valid_targets
    impute_data = df[use_cols].copy()
    
    # KNN 실행
    print(f" -> 로직: {len(impute_data)}개 데이터에 대해 KNN(k={k}) 수행. 타겟: {valid_targets}")
    imputer = KNNImputer(n_neighbors=k)
    out_data = imputer.fit_transform(impute_data)
    
    # 결과 적용
    out_df = pd.DataFrame(out_data, columns=use_cols, index=impute_data.index)
    for col in valid_targets:
        df[col] = out_df[col]
        
    print(" -> 완료.")
    return df

print("--- 학습 데이터 보완 ---")
df_train = run_knn_imputation(df_train)
print("--- 테스트 데이터 보완 ---")
df_test = run_knn_imputation(df_test)

## 최종 결과 저장

In [ ]:
# 저장 경로 설정 (data/processed 폴더)
SAVE_DIR = os.path.join(DATA_DIR, 'processed')
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)
    
train_save = os.path.join(SAVE_DIR, 'train_final.csv')
test_save = os.path.join(SAVE_DIR, 'test_final.csv')

df_train.to_csv(train_save, index=False)
df_test.to_csv(test_save, index=False)

print("모든 전처리 과정이 완료되었습니다!")
print(f"저장된 경로:\n  {train_save}\n  {test_save}")